In [6]:
from pydantic import BaseModel
from torch import FloatTensor, LongTensor


class Data(BaseModel):
    laser_params: FloatTensor
    emiss: FloatTensor
    uids: LongTensor
    wavelength: FloatTensor

    class Config:
        arbitrary_types_allowed = True


In [25]:
from models.forward_model import ForwardModel
from models.backward_model import BackwardModel
from models.base_model import BaseModel
from config import Config
from models.model_factory import ModelFactory
from pathlib import Path
from einops.layers.torch import Rearrange
from typing import Optional

import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader, TensorDataset

import utils
from config import Config
from dto.data import Data
from utils import FileUtils, split
from utils.utils import Stage

# CKPT_PATH = "../data/models/model.ckpt" # "../weights/forward/epoch=0122-step=1230-val_loss=0.28432.ckpt"
# MODEL_PATH = "data/mymodels"
#load data
config = Config()

data_file = Path(f"../{config.data_folder}/{config.data_file}")
data = torch.load(data_file)
data_struct = Data(
        laser_params=data["normalized_laser_params"],
        emiss=data["interpolated_emissivity"],
        uids=data["uids"],
        wavelength=data["wavelength"],
    )
    # Set the number of wavelengths
    # config.num_wavelens = data["interpolated_emissivity"].shape[-1]
#     return Data(
#         laser_params=data["normalized_laser_params"],
#         emiss=data["interpolated_emissivity"],
#         uids=data["uids"],
#         wavelength=data["wavelength"],
#     )

# data = read_pt_data(config.data_folder, config.data_file)
splits = split(len(data_struct.laser_params))
train, val, test = [
    TensorDataset(
        data_struct.laser_params[splits[s].start : splits[s].stop],
        data_struct.emiss[splits[s].start : splits[s].stop],
        data_struct.uids[splits[s].start : splits[s].stop],
    )
    for s in ("train", "val", "test")
]


In [26]:

import shutil
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader, random_split

class LambdaLayer(nn.Module):
    
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd
    
    def forward(self, x):
        return self.lambd(x)

class BasicConvBlock(nn.Module):
    
    ''' The BasicConvBlock takes an input with in_channels, applies some blocks of convolutional layers 
    to reduce it to out_channels and sum it up to the original input. 
    If their sizes mismatch, then the input goes into an identity. 
    
    Basically The BasicConvBlock will implement the regular basic Conv Block + 
    the shortcut block that does the dimension matching job (option A or B) when dimension changes between 2 blocks
    '''
    
    def __init__(self, in_channels, out_channels, stride=1, option='A'):
        super(BasicConvBlock, self).__init__()
        
        self.features = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)),
            ('bn1', nn.BatchNorm2d(out_channels)),
            ('act1', nn.ReLU()),
            ('conv2', nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn2', nn.BatchNorm2d(out_channels))
        ]))

        self.shortcut = nn.Sequential()
        
        '''  When input and output spatial dimensions don't match, we have 2 options, with stride:
            - A) Use identity shortcuts with zero padding to increase channel dimension.    
            - B) Use 1x1 convolution to increase channel dimension (projection shortcut).
         '''
        if stride != 1 or in_channels != out_channels:
            if option == 'A':
                # Use identity shortcuts with zero padding to increase channel dimension.
                pad_to_add = out_channels//4
                ''' ::2 is doing the job of stride = 2
                F.pad apply padding to (W,H,C,N).
                
                The padding lengths are specified in reverse order of the dimensions,
                F.pad(x[:, :, ::2, ::2], (0,0, 0,0, pad,pad, 0,0))

                [width_beginning, width_end, height_beginning, height_end, channel_beginning, channel_end, batchLength_beginning, batchLength_end ]

                '''
                self.shortcut = LambdaLayer(lambda x:
                            F.pad(x[:, :, ::2, ::2], (0,0, 0,0, pad_to_add, pad_to_add, 0,0)))
            if option == 'B':
                self.shortcut = nn.Sequential(OrderedDict([
                    ('s_conv1', nn.Conv2d(in_channels, 2*out_channels, kernel_size=1, stride=stride, padding=0, bias=False)),
                    ('s_bn1', nn.BatchNorm2d(2*out_channels))
                ]))
        
    def forward(self, x):
        out = self.features(x)
        # sum it up with shortcut layer
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [27]:
class ResNet(nn.Module):
    """
        ResNet-56 architecture for CIFAR-10 Dataset of shape 32*32*3
    """
    def __init__(self, block_type, num_blocks, in_channels=14, out_channels=512, kernel_size=1, stride=1, padding=1):
        super(ResNet, self).__init__()
        
        self.in_channels = in_channels
        
        self.conv0 = nn.Conv2d(in_channels, 16, kernel_size=kernel_size, stride=1, padding=1, bias=False)
        self.bn0 = nn.BatchNorm2d(16)
        
        self.block1 = self.__build_layer(block_type, 16, num_blocks[0], starting_stride=1)
        
        self.block2 = self.__build_layer(block_type, 32, num_blocks[1], starting_stride=2)
        
        self.block3 = self.__build_layer(block_type, 64, num_blocks[2], starting_stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.linear = nn.Linear(64, 10)
    
    def __build_layer(self, block_type, out_channels, num_blocks, starting_stride):
        
        strides_list_for_current_block = [starting_stride] + [1]*(num_blocks-1)
        ''' Above line will generate an array whose first element is starting_stride
        And it will have (num_blocks-1) more elements each of value 1
         '''
        # print('strides_list_for_current_block ', strides_list_for_current_block)
        
        layers = []
        
        for stride in strides_list_for_current_block:
            layers.append(block_type(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn0(self.conv0(x)))
        out = self.block1(out)
        out = self.block2(out)        
        out = self.block3(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.linear(out)
        return out

In [30]:
model = nn.Sequential(
        Rearrange("b c -> b c 1 1"),
        ResNet(block_type=BasicConvBlock, num_blocks=[9,9,9], in_channels=14, out_channels=512),
        nn.Sigmoid()
        )
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# device = 'cpu'
model.to(device)
summary(model, (3, 32, 32))

EinopsError: Expected 2 dimensions, got 4

In [29]:
EPOCHS = 2
train_samples_num = 45000
val_samples_num = 5000
train_costs, val_costs = [], []
train_loader = DataLoader(train, batch_size=1000, shuffle=True)
val_loader = DataLoader(val, batch_size=1000, shuffle=True)
test_loader = DataLoader(test, batch_size=1000, shuffle=True)
    
#Training phase.    
for epoch in range(EPOCHS):

    train_running_loss = 0
    correct_train = 0    
    model.train().cuda()
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        """ for every mini-batch during the training phase, we typically want to explicitly set the gradients 
        to zero before starting to do backpropragation """
        optimizer.zero_grad()
        
        # Start the forward pass
        prediction = model(inputs)
                    
        loss = criterion(prediction, labels)
        
        # do backpropagation and update weights with step()
        loss.backward()         
        optimizer.step()
        
        # print('outputs on which to apply torch.max ', prediction)
        # find the maximum along the rows, use dim=1 to torch.max()
        _, predicted_outputs = torch.max(prediction.data, 1)
        
        # Update the running corrects 
        correct_train += (predicted_outputs == labels).float().sum().item()
        
        ''' Compute batch loss
        multiply each average batch loss with batch-length. 
        The batch-length is inputs.size(0) which gives the number total images in each batch. 
        Essentially I am un-averaging the previously calculated Loss '''
        train_running_loss += (loss.data.item() * inputs.shape[0])


    train_epoch_loss = train_running_loss / train_samples_num
    
    train_costs.append(train_epoch_loss)
    
    train_acc =  correct_train / train_samples_num

    # Now check trained weights on the validation set
    val_running_loss = 0
    correct_val = 0
    
    model.eval().cuda()
a
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass.
            prediction = model(inputs)

            # Compute the loss.
            loss = criterion(prediction, labels)

            # Compute validation accuracy.
            _, predicted_outputs = torch.max(prediction.data, 1)
            correct_val += (predicted_outputs == labels).float().sum().item()

        # Compute batch loss.
        val_running_loss += (loss.data.item() * inputs.shape[0])

        val_epoch_loss = val_running_loss / val_samples_num
        val_costs.append(val_epoch_loss)
        val_acc =  correct_val / val_samples_num
    
    info = "[Epoch {}/{}]: train-loss = {:0.6f} | train-acc = {:0.3f} | val-loss = {:0.6f} | val-acc = {:0.3f}"
    
    print(info.format(epoch+1, EPOCHS, train_epoch_loss, train_acc, val_epoch_loss, val_acc))
    
    torch.save(model.state_dict(), '/content/checkpoint_gpu_{}'.format(epoch + 1)) 
                                                            
torch.save(model.state_dict(), '/content/resnet-56_weights_gpu')  
    
return train_costs, val_costs

IndentationError: unexpected indent (3561369123.py, line 58)